In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import h5py
from pathlib import Path

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors
import seaborn as sns

import flammkuchen as fl
from scipy.interpolate import interp1d

C:\Users\admin\AppData\Local\Temp\ipykernel_18188\1709915033.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [19]:
from mb_helper import compute_angle_between_vect_tail, compute_angle_between_vect, exptrapolate_segments, mid_head, midpoint
from mb_helper import tail_angles, fin_preprocess, calculate_angles, compute_body_angle
from mb_helper import labels_cat, color

In [20]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Load Bouts

In [21]:
master_path = Path(Path(r'\\portulab.synology.me\data\Kata\Figures_Paper\Koetter-et-al_data\Testdata') / 'Raw_Data')
out_path = Path(master_path/'Processed_Data')
fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[WindowsPath('//portulab.synology.me/data/Kata/Figures_Paper/Koetter-et-al_data/Testdata/Raw_Data/240423_f0')]

In [22]:
fish= 0
fish_path = fish_paths[fish]
fish_id =  fish_paths[fish].name#[:-13]
exp_name = Path(fish_paths[fish]).parts[-2]
exp_name = 'testfish'
exp_name, fish_id

('testfish', '240423_f0')

### Load DLC

In [23]:
### Load and replace eye data
#t, left eye [0], right eye [1], endpoints of eye vector end points (point1xy, point2xy) for anterior and posterior
eye_coords = fl.load(fish_path/'eye_coords.h5')['eye_coords']
eye_coords.shape

(95423, 2, 2, 2)

In [24]:
left_eye_points = np.asarray(eye_coords)[:,0]
right_eye_points = np.asarray(eye_coords)[:,1]

l_anterior = np.asarray(left_eye_points[:,0])
l_posterior = np.asarray(left_eye_points[:,1])

left_eye_anterior_x = []
left_eye_anterior_y = []
left_eye_posterior_x = []
left_eye_posterior_y = []
for i in range(l_anterior.shape[0]):
    left_eye_anterior_x.append(l_anterior[i][0])
    left_eye_anterior_y.append(l_anterior[i][1])
    left_eye_posterior_x.append(l_posterior[i][0])
    left_eye_posterior_y.append(l_posterior[i][1])
    
r_anterior = np.asarray(right_eye_points[:,0])
r_posterior = np.asarray(right_eye_points[:,1])

right_eye_anterior_x = []
right_eye_anterior_y = []
right_eye_posterior_x = []
right_eye_posterior_y = []
for i in range(r_anterior.shape[0]):
    right_eye_anterior_x.append(r_anterior[i][0])
    right_eye_anterior_y.append(r_anterior[i][1])
    right_eye_posterior_x.append(r_posterior[i][0])
    right_eye_posterior_y.append(r_posterior[i][1])
    

### Load DLC and preprocess

In [25]:
fps=200
mm_per_unit = 1/70
N_seg = 10
N = eye_coords.shape[0]

In [26]:

df = pd.read_csv(out_path/ '{}_DLC.csv'.format(fish_id), header=[0,1])
print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')
print('working on {} frames'.format(df.shape[0]))

#Extract angles
body_x = df.body.values[:, 0].astype('float')
body_y = df.body.values[:, 1].astype('float')

tail_x_col = [f'tail_{i}' for i in range(5)]
tail_y_col = [f'tail_{i}' for i in range(5)]
tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])

# upsample tail to 10 segments
tail_x_10, tail_y_10 = exptrapolate_segments(tail_x, tail_y, N_seg)


7.9519166666666665 minutes at 200 fps
working on 95423 frames


In [27]:
tail_x_10.shape

(11, 95423)

In [28]:
## replace eyes and recompute mid head

In [29]:
mid_headx, mid_heady, left_mid_eye_x, left_mid_eye_y, right_mid_eye_x, right_mid_eye_y = mid_head(
             np.asarray(right_eye_posterior_x), np.asarray(right_eye_posterior_y),
             np.asarray(right_eye_anterior_x), np.asarray(right_eye_anterior_y),
             np.asarray(left_eye_posterior_x), np.asarray(left_eye_posterior_y),
             np.asarray(left_eye_anterior_x), np.asarray(left_eye_anterior_y))


In [36]:
head_x = mid_headx
head_y = mid_heady

In [37]:
head_x.shape, head_y.shape

((95423,), (95423,))

### compute body angle

In [43]:
angles_radians, body_angle = compute_body_angle(head_x, head_y, body_x, body_y)

In [47]:
left_fin_vect, right_fin_vect, left_fin_angle, right_fin_angle = fin_preprocess(df, angles_radians, mid_headx, mid_heady, tail_x_10, tail_y_10)

corr fins


In [59]:
fl.save(master_path/'body_angle.h5',  angles_radians)
fl.save(master_path/'fin_angles.h5',  np.asarray([left_fin_angle, right_fin_angle]).T)